# Recommendation system(Lesson 1)
## Content based recommender

### First of all import needed packages

In [1]:
from math import sqrt
import pandas as pd
import numpy as np
# Use this to show matplotlib and plots properly(But not essential)
%matplotlib inline

### Read the csv(Comma seperated value) file with pandas

In [2]:
Movies_data_frame = pd.read_csv('Movies.csv')
Ratings_data_frame = pd.read_csv('Ratings.csv')
Movies_data_frame.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


### Cleaning data

In [3]:
# Findind the years in parantheses(not those which are in the movie's names)
Movies_data_frame['year'] = Movies_data_frame.title.str.extract(r'\((\d{4})\)',expand=False)
# Removing the parentheses
Movies_data_frame['year'] = Movies_data_frame.year.str.extract(r'(\d{4})',expand=False)
# Removing the years from the title column
Movies_data_frame['title'] = Movies_data_frame.title.str.replace(r'\(\d{4}\)','',regex=True)
# The strip function : to get rid of any ending whitespace characters that may have appeared
Movies_data_frame['title'] = Movies_data_frame['title'].apply(lambda x : x.strip())
Movies_data_frame.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [4]:
# Removing pipe lines
Movies_data_frame['genres'] = Movies_data_frame.genres.str.split('|')
Movies_data_frame.head()

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


In [5]:
Movies_data_frame_with_genres = Movies_data_frame.copy()
# For every row in the data frame, iterate through the list of genres and place 1 into the corresponding column
for index , row in Movies_data_frame.iterrows() :
    for genre in row['genres'] :
        Movies_data_frame_with_genres.at[index,genre] = 1
# Shows that a movie doesn't have that column's genre , set 0 for that(filling Nan with 0)
Movies_data_frame_with_genres = Movies_data_frame_with_genres.fillna(0)
Movies_data_frame_with_genres.head()

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,[Comedy],1995,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [122]:
Ratings_data_frame.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [123]:
# Removing the column timestamp
Ratings_data_frame = Ratings_data_frame.drop('timestamp',axis=1)
Ratings_data_frame.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [124]:
User_input = [
            {'title':'Breakfast Club, The','rating':5},
            {'title':'Toy Story','rating':3.5},
            {'title':'Jumanji','rating':2},
            {'title':"Pulp Fiction",'rating':5},
            {'title':'Akira','rating':4.5}
         ]
Input_movies_data_frame = pd.DataFrame(User_input)
Input_movies_data_frame.head()

,title,rating
0,"Breakfast Club, The",5.0
1,Toy Story,3.5
2,Jumanji,2.0
3,Pulp Fiction,5.0
4,Akira,4.5


In [125]:
# Filtering out the movies by title
Input_id = Movies_data_frame[Movies_data_frame['title'].isin(Input_movies_data_frame['title'].tolist())]
Input_movies_data_frame = pd.merge(Input_id,Input_movies_data_frame)
Input_movies_data_frame = Input_movies_data_frame.drop('genres',axis=1).drop('year',axis=1)
Input_movies_data_frame.head()

,movieId,title,rating
0,1,Toy Story,3.5
1,2,Jumanji,2.0
2,296,Pulp Fiction,5.0
3,1274,Akira,4.5
4,1968,"Breakfast Club, The",5.0


In [126]:
# Filtering out the movies from the input
User_movies = Movies_data_frame_with_genres[Movies_data_frame_with_genres['movieId'].isin(Input_movies_data_frame['movieId'].tolist())]
User_movies

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
257,296,Pulp Fiction,"[Comedy, Crime, Drama, Thriller]",1994,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
973,1274,Akira,"[Action, Adventure, Animation, Sci-Fi]",1988,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1445,1968,"Breakfast Club, The","[Comedy, Drama]",1985,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [127]:
# reset_index() : start indices from 1
User_movies = User_movies.reset_index(drop=True)
User_genre_table = User_movies.drop('movieId',axis=1).drop('title',axis=1).drop('genres',axis=1).drop('year',axis=1)
User_genre_table.head()

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [128]:
Input_movies_data_frame['rating']

0    3.5
1    2.0
2    5.0
3    4.5
4    5.0
Name: rating, dtype: float64

In [129]:
# Dot produt to get weights
User_profile = User_genre_table.transpose().dot(Input_movies_data_frame['rating'])
User_profile

Adventure             10.0
Animation              8.0
Children               5.5
Comedy                13.5
Fantasy                5.5
Romance                0.0
Drama                 10.0
Action                 4.5
Crime                  5.0
Thriller               5.0
Horror                 0.0
Mystery                0.0
Sci-Fi                 4.5
War                    0.0
Musical                0.0
Documentary            0.0
IMAX                   0.0
Western                0.0
Film-Noir              0.0
(no genres listed)     0.0
dtype: float64

In [130]:
Genre_table = Movies_data_frame_with_genres.set_index(Movies_data_frame_with_genres['movieId'])
Genre_table = Genre_table.drop('movieId',axis=1).drop('title',axis=1).drop('genres',axis=1).drop('year',axis=1)
Genre_table.head()

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [131]:
Genre_table.shape

(9742, 20)

### Make recommendations

In [132]:
# Multiply the genres by the weights and then take the weighted average
Recommendation_table_data_frame = ((Genre_table*User_profile).sum(axis=1)/(User_profile.sum()))
Recommendation_table_data_frame.head()

movieId
1    0.594406
2    0.293706
3    0.188811
4    0.328671
5    0.188811
dtype: float64

In [133]:
Recommendation_table_data_frame = Recommendation_table_data_frame.sort_values(ascending=False)
Recommendation_table_data_frame.head(100)

movieId
134853    0.734266
148775    0.685315
117646    0.678322
6902      0.678322
81132     0.671329
            ...   
145       0.531469
5628      0.531469
4818      0.531469
2761      0.531469
82765     0.531469
Length: 100, dtype: float64

In [134]:
Movies_data_frame.loc[Movies_data_frame['movieId'].isin(Recommendation_table_data_frame.head(20).keys())]

,movieId,title,genres,year
559,673,Space Jam,"[Adventure, Animation, Children, Comedy, Fanta...",1996
1390,1907,Mulan,"[Adventure, Animation, Children, Comedy, Drama...",1998
2250,2987,Who Framed Roger Rabbit?,"[Adventure, Animation, Children, Comedy, Crime...",1988
3460,4719,Osmosis Jones,"[Action, Animation, Comedy, Crime, Drama, Roma...",2001
4631,6902,Interstate 60,"[Adventure, Comedy, Drama, Fantasy, Mystery, S...",2002
5490,26340,"Twelve Tasks of Asterix, The (Les douze travau...","[Action, Adventure, Animation, Children, Comed...",1976
5819,32031,Robots,"[Adventure, Animation, Children, Comedy, Fanta...",2005
6047,40339,Chicken Little,"[Action, Adventure, Animation, Children, Comed...",2005
6448,51939,TMNT (Teenage Mutant Ninja Turtles),"[Action, Adventure, Animation, Children, Comed...",2007
6455,52287,Meet the Robinsons,"[Action, Adventure, Animation, Children, Comed...",2007


In [142]:
K = Movies_data_frame.loc[Movies_data_frame['movieId'].isin(Recommendation_table_data_frame.head(20).keys())].reset_index()
for i in range(len(K['title'])) :
    print(f'Recommendation {i+1} : {K['title'][i]} | with movie id : {K['movieId'][i]}')

Recommendation 1 : Space Jam | with movie id : 673
Recommendation 2 : Mulan | with movie id : 1907
Recommendation 3 : Who Framed Roger Rabbit? | with movie id : 2987
Recommendation 4 : Osmosis Jones | with movie id : 4719
Recommendation 5 : Interstate 60 | with movie id : 6902
Recommendation 6 : Twelve Tasks of Asterix, The (Les douze travaux d'Astérix) | with movie id : 26340
Recommendation 7 : Robots | with movie id : 32031
Recommendation 8 : Chicken Little | with movie id : 40339
Recommendation 9 : TMNT (Teenage Mutant Ninja Turtles) | with movie id : 51939
Recommendation 10 : Meet the Robinsons | with movie id : 52287
Recommendation 11 : Futurama: Bender's Game | with movie id : 62956
Recommendation 12 : Rubber | with movie id : 81132
Recommendation 13 : Mars Needs Moms | with movie id : 85261
Recommendation 14 : Ernest & Célestine (Ernest et Célestine) | with movie id : 108540
Recommendation 15 : The Lego Movie | with movie id : 108932
Recommendation 16 : Dragonheart 2: A New Begi